# Charged Patchy Particle Model
_Alexei Abrikossov and Mikael Lund, December 2016_

In this Notebook we setup an MC simulation to calculate the interaction free energy between a pair
of CPPM's.

In [ ]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

import numpy as np
from scipy import integrate
import pandas as pd
import os.path, os, sys, json, shutil
from pathlib import Path
import pickle
from sys import stdout
from math import exp, sqrt
import mdtraj as md

plt.rcParams.update({'font.size': 16, 'figure.figsize': [6.0, 5.0]})
try:
    workdir
except NameError:
    workdir=%pwd
else:
    %cd $workdir

### Download and compile the MC software Faunus

In [ ]:
%%bash -s "$workdir"
cd $1
# if different, copy custom gctit.cpp into faunus
if ! cmp mc/twobody.cpp faunus/src/examples/twobody.cpp >/dev/null 2>&1
then
    cp mc/twobody.cpp faunus/src/examples/
fi

if [ ! -d "faunus" ]; then
  git clone https://github.com/mlund/faunus.git
  cd faunus
  git checkout 47910f92f1acbc5b4dd6815c533e868f2a4478a7
else
  cd faunus
fi
CXX=g++-mp-5 CC=gcc-mp-5 cmake . -DCMAKE_BUILD_TYPE=Release -DENABLE_APPROXMATH=on -DENABLE_OPENMP=on &>/dev/null
make example_twobody -j
cd $1

### Function for generating JSON input file for Faunus

In [ ]:
import os.path, os, sys, json, shutil
from pathlib import Path

def mkinput(ions):
    js = {
          "energy" : {
            "nonbonded" : {
              "coulomb" : { "epsr" : 78.7, "ionicstrength" : 0.1 },
              "lj" : {
                    "custom" : {
                        "MP Na": dict(sigma=(40+2)/2.0, eps=0.01*2.5),
                        "MP La": dict(sigma=(40+2)/2.0, eps=0.01*2.5),
                        "MP Cl": dict(sigma=(40+2)/2.0, eps=0.01*2.5)
                      }
                    }
            },

            "cmconstrain" : {
              "sphere1 sphere2" : { "mindist": 0, "maxdist": maxdist }
              }
            },

          "atomlist" : {
            "UP":  dict(q=0,  sigma=3.0, eps=0.2479, mw=1e-3),
            "NP":  dict(q=-1, sigma=3.0, eps=0.2479, mw=1e-3),
            "PP":  dict(q=1,  sigma=3.0, eps=0.2479, mw=1e-3),
            "MP":  dict(q=0,  sigma=40,  eps=0,      mw=1e6),
            "Na":  dict(q=1,  sigma=2.0, eps=0.2479, mw=1e-3, dp=50),
            "La":  dict(q=3,  sigma=2.0, eps=0.2479, mw=1e-3, dp=20),
            "Cl":  dict(q=-0, sigma=2.0, eps=0.2479, mw=1e-3, dp=50)
              },

          "moleculelist": {
              "0sphere":  { "structure":xyzfile, "Ninit":1, "insdir":"0 0 0", "insoffset":"0 0 "+str(offset)},
              "1sphere":  { "structure":xyzfile, "Ninit":1, "insdir":"0 0 0", "insoffset":"0 0 -"+str(offset)}
              },

          "moves" : {
              "moltransrot2body" : {
                "0sphere" : { "dp":dp, "dprot":1 }, 
                "1sphere" : { "dp":dp, "dprot":1 } 
                }
          },
          "analysis" : {
            "pqrfile" : { "file": "confout.pqr"  },
            "statefile" : { "file": "state" },
            "xtcfile" : { "file": "traj.xtc", "nstep": nstep_xtc }
          },

          "system" : {
              "temperature" : 298.15,
              "cylinder" : { "length" : 120, "radius" : 50 },
              "mcloop"   : { "macro" : 10, "micro" : micro }
              }
          }
    
    for name, N in ions.items():
        if N>0:
            js['moleculelist'][name] = dict(Ninit=N, atomic=True, atoms=name)
            if not 'atomtranslate' in js['moves']:
                js['moves']['atomtranslate'] = {}
            js['moves']['atomtranslate'][name] = dict(peratom=True)

    with open('twobody.json', 'w+') as f:
        f.write(json.dumps(js, indent=4))
        
offset=40   # initial COM-COM separation (angstrom)
maxdist=500 # maximum allowed COM-COM distance (angstrom)
dp=4        # COM translational displacement parameter (angstrom)
nstep_xtc=0 # frequency for saving frames to xtc trajectory file

## Potential of mean force from histogram method

We first calculate the potential of mean force between two neutral spheres by simply sampling the COM-COM distance probability distribution, followed by Boltzmann inversion. Later we can check this result by comparing with PMFs obtained from direct sampling of the mean force from simulations where the particles are kept at fixed positions.

The COM's of the two macromolecules are able to translate along a line which coinsides with the axis of a cylindrical simulation cell with hard boundaries. During simulation, the molecules further rotate around their mass centers. Since the molecules in this way are constrained on a line, there is no need to correct for the increasing volume element normally needed for simulations in free space.

In [ ]:
%cd $workdir/mc

if not os.path.isfile('histrdf.dat'):
    
    xyzfile='sphere-neutral.xyz' # neutral sphere, 600 atoms
    offset=50/2.0                # COM offset from origo (center of container)
    dp=6                         # protein displacement on line (angstrom)
    micro=100                    # number of micro steps (equilibration)
    ions={'Na':0}                # we don't want any ions right now
    mkinput( ions )              # make json input file for faunus

    !rm -fR state                # make sure there's no old state (restart) file
    !OMP_NUM_THREADS=4 ../faunus/src/examples/twobody &> eq_hist # eq. run

    micro=20000                  # number of micro steps for production
    nstep_xtc=200                # save xtc file
    mkinput( ions )
    !OMP_NUM_THREADS=4 ../faunus/src/examples/twobody > out_hist # production run.
    !cp -fR rdf.dat histrdf.dat
    nstep_xtc=0                  # disable xtc output for future simulations

    print('done.')
    
# Boltzmann inversion and plot
r_hist, g_hist = np.loadtxt('histrdf.dat', unpack=True)
w_hist = -np.log( g_hist / g_hist[r_hist>50].mean() )

plt.plot(r_hist, w_hist, label='MC, histogram' )
plt.xlabel('$R$ (Å)')
plt.ylabel('PMF ($k_BT$)')
plt.xlim(41, 50)
plt.legend(loc=0, frameon=False)

print('PMF minimum =', w_hist[r_hist<60].min(), 'kT')

### Visualize trajectory (unstable)

In [ ]:
import nglview as nv
traj = md.load_xtc('traj.xtc', top='confout.pdb')
view = nv.show_mdtraj(traj)
view.add_spacefill(selection='all')
view

## Mean force calculation for fixed separations

In multivalent electrolyte solution, highly charged ion may "stick" to the molecule surface, making translational and rotational moves diffucult (low acceptance). To counter this, we sample the PMF by calculating for mean force on the macro-particles at fixed separations. The PMF is obtained by subsequent integration.

An automated setup is used and in the following cell, we create a Pandas structure describing how the simulations should be run and plotted (number of ions, macro-particle structure, colors, etc).

In [ ]:
R_range = np.arange(42, 70, 1) # separation space to scan (angstrom)
param = pd.DataFrame({
    'Neutral' :
        dict(
            charge=0, xyzfile=workdir+'/mc/sphere-neutral.xyz',
            R=R_range, label=r'Neutral', color='black',
            ions = dict(Na=0)
        ),
    'P00-mono' :
        dict(
            charge=-8, xyzfile=workdir+'/mc/sphere-P00.xyz',
            R=R_range, label=r'$P_0^0$ (Na$^+$)', color='blue',
            ions = dict(Na=16)
        ),
    'P00-tri' :
        dict(
            charge=-8, xyzfile=workdir+'/mc/sphere-P00.xyz',
            R=R_range, label=r'$P_0^0$ (La$^{3+}$)', color='green',
            ions = dict(La=5, Na=1)
    ),
    'P18' :
        dict(
            charge=-8, xyzfile=workdir+'/mc/sphere-P18.xyz',
            R=R_range, label=r'$P_8^1$', color='orange',
            ions = [ dict(Na=16), dict(La=5, Na=1) ]
        )
    })
param = param.T.drop(labels='P18').T
#param = param.T.drop(labels='Neutral').T
#param = param.T.drop(labels='P00-tri').T
param

### Loop over all defined systems and simulate

In [ ]:
%%writefile $workdir/mc/submit.sh
# This is a submit file, should you be using a cluster
#!/bin/bash
#SBATCH -N 1
#SBATCH -n 1
#SBATCH -t 01:00:00
../../faunus/src/examples/twobody > out

In [ ]:
%cd -q $workdir/mc
for particletype, d in param.items(): # 'P00', 'P18', ...

        ionstr = ''
        for ion, N in d.ions.items(): # salt particles
            ionstr = ion+str(N) + ionstr

        for R in d.R:  # loop over defined COM-COM separations

            directory = particletype + '-' + ionstr + '-R' + str(R)
            if not os.path.isdir(directory):
                %mkdir $directory
            %cd $directory

            dp=0           # macromolecules do not move
            offset = R/2.0 # initial COM distance from origo
            micro=100
            xyzfile = d.xyzfile
            mkinput( d.ions )

            !rm -fR state
            !OMP_NUM_THREADS=4 nice ../../faunus/src/examples/twobody &> eq # equilibration

            micro=1000     # number of micro steps in production
            mkinput( d.ions )

            if shutil.which('sbatch') is not None: # run on slurm cluster...
                !sbatch ../submit.sh
            else:                                  # ...or locally (slow) ?
                !OMP_NUM_THREADS=4 ../../faunus/src/examples/twobody > out

            %cd -q ..
print('done.')

### Collect; analyse; and plot data

In [ ]:
%cd -q $workdir/mc
for particletype, d in param.items(): # 'P00', 'P18', ...
    
        ionstr = ''
        for ion, N in d.ions.items(): # loop over all ion conditions
            ionstr = ion+str(N) + ionstr

        F = []         # mean force vs. distance
        for R in d.R:  # loop over defined COM-COM separations

            directory = particletype + '-' + ionstr + '-R' + str(R)
            if os.path.isdir(directory):
                %cd -q $directory
                with open('analysis_out.json') as file:    # simulation analysis is saved here...
                    j = json.load(file)['Mean force']['meanforce']
                    mf = np.array( j ).mean() # mean force on group 1 and 2
                    F.append( [R,mf]  )

                %cd -q ..

        r, f = np.array(F).T                      # r=distance (angstrom), f=mean force (kT/angstrom)
        w = integrate.cumtrapz( f, r, initial=0 ) # integrate force --> potential of mean force, w
        plt.plot(r, w-w[-1], label=d.label, color=d.color)

plt.plot(r_hist, w_hist, 'k--', ms=3, label='Neutral, histogram')
plt.legend(loc=0, frameon=False)
plt.xlabel('$R$ (Å)')
plt.ylabel('PMF ($k_BT$)')
plt.xlim(42,70)